In [ ]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [ ]:
pip install citipy

In [5]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [6]:
city = citipy.nearest_city(22.99,120.21)
city

In [7]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

amderma ru
komsomolskiy ru
busselton au
dakar sn
umzimvubu za
kaitangata nz
balabac ph
tiksi ru
punta arenas cl
hamilton bm
oranjestad aw
sorvag fo
nikolayevskaya ru
opuwo na
denpasar id
rikitea pf
wa gh
ancud cl
punta arenas cl
san policarpo ph
ushuaia ar
olinda br
havre-saint-pierre ca
yellowknife ca
rikitea pf
airai pw
busselton au
tessalit ml
punta arenas cl
san pedro ar
mehamn no
catuday ph
ancud cl
esperance au
potosi bo
rikitea pf
belushya guba ru
taolanaro mg
bredasdorp za
qandala so
arkhangelsk ru
rikitea pf
coari br
cheney us
torbay ca
grahamstown za
jamestown sh
kamiiso jp
sicamous ca
yenisea gr
cape town za
tazovskiy ru
castro cl
dunedin nz
victoria sc
rikitea pf
yellowknife ca
taolanaro mg
jamestown sh
tianpeng cn
yafran ly
carauari br
albany au
manadhoo mv
brownsville us
riberalta bo
mingyue cn
port elizabeth za
busselton au
bredasdorp za
ushuaia ar
voskresenskoye ru
atuona pf
nikolskoye ru
rikitea pf
asau tv
port keats au
samarai pg
busselton au
puerto ayora ec
punta are

cape town za
cockburn town tc
nome us
puerto ayora ec
vaini to
nanortalik gl
vila velha br
bambous virieux mu
naze jp
matara lk
pangai to
shawville ca
richards bay za
sola vu
halifax ca
cape town za
atuona pf
deputatskiy ru
balaipungut id
krasnyy luch ru
manggar id
los llanos de aridane es
bandarbeyla so
noumea nc
albany au
ilhabela br
taolanaro mg
sumbawa id
castro cl
hasaki jp
havre us
katsuura jp
morant bay jm
hithadhoo mv
bonthe sl
punta arenas cl
punta arenas cl
lodja cd
qaanaaq gl
hithadhoo mv
hofn is
broome au
iglesias it
dingle ie
talnakh ru
albany au
kodiak us
hobart au
cidreira br
puerto ayora ec
bambous virieux mu
naron es
thompson ca
rikitea pf
hilo us
ushuaia ar
esperance au
iqaluit ca
rikitea pf
norman wells ca
cape town za
airai pw
aksarka ru
labutta mm
aras no
bintulu my
ushuaia ar
causapscal ca
geraldton ca
saldanha za
laguna br
busselton au
ylivieska fi
ushuaia ar
provideniya ru
butaritari ki
la quinta us
husavik is
cape town za
hasaki jp
mahebourg mu
nikolskoye ru
pe

arraial do cabo br
cherskiy ru
butaritari ki
arraial do cabo br
yakeshi cn
mys shmidta ru
slave lake ca
sao filipe cv
sept-iles ca
airai pw
busselton au
busselton au
vardo no
puerto ayora ec
kavieng pg
tiksi ru
ushuaia ar
sitka us
mayo ca
taolanaro mg
dingle ie
sinnamary gf
lakes entrance au
nome us
pevek ru
busselton au
ceuta es
north bend us
ushuaia ar
ushuaia ar
grand river south east mu
esperance au
imeni poliny osipenko ru
rikitea pf
east london za
palmer us
deputatskiy ru
nizhneyansk ru
ahipara nz
salalah om
khandyga ru
mataura pf
rikitea pf
cape town za
lishui cn
kruisfontein za
hermanus za
kapaa us
yatou cn
grants pass us
tabiauea ki
vanimo pg
douentza ml
hithadhoo mv
port elizabeth za
bethel us
kapaa us
puerto ayora ec
ankang cn
gorno-chuyskiy ru
luderitz na
rikitea pf
marshfield us
luderitz na
mariental na
puerto ayora ec
samusu ws
avarua ck
huilong cn
bur gabo so
cape town za
louisbourg ca
bac lieu vn
komsomolskiy ru
vila franca do campo pt
grand centre ca
sumenep id
tecoana

In [8]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

632